In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida

import ipywidgets as ipw
from aiida.engine import run
from IPython.display import clear_output

from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, ProcessFollowerWidget
from aiidalab_widgets_base import ProgressBarWidget, viewer

NetworkCalculation = CalculationFactory('zeopp.network')

In [ ]:
def on_click_submit():
    builder = NetworkCalculation.get_builder()
    
    builder.parameters = DataFactory('zeopp.parameters')(
        dict={
            'ha': 'DEF',  # Just for speed; use 'DEF' for prodution!
            'cssr': True,  # Converting to cssr.
            'res': True,
            'sa': [probe_radius.value, probe_radius.value, 100000],  # Compute surface area.
            'volpo': [probe_radius.value, probe_radius.value, 100000],  # Compute geometric pore volume.
            'block': [probe_radius.value, 100]  # Compute blocking spheres for a big molecule.
        })
    output.value = ''
    if structure_widget.structure_node is None:
        output.value = "Please upload a structure first"
        return None
    builder.structure = structure_widget.structure_node

    if code_dropdown.selected_code is None:
        output.value =  "Please select a code first"
        return None
    builder.code = code_dropdown.selected_code
    
    builder.metadata.label = "ZeoppPoreAnalysis"
    
    return builder

def show_results(process):
    output = ipw.HTML("""<hr>
    <h3> Step 4: Get your results </h3>""")
    display(output, viewer(process.inputs.structure), viewer(process.outputs.output_parameters))
    
def follow_process(process):
    follower = ProcessFollowerWidget(process, followers=[ProgressBarWidget()])
    follower.on_completed(show_results)
    display(follower)
    follower.follow(detach=True)

## Step 1: Select structure

Before uploading, please
 * **Desolvate** the structure
 * Use **P1 symmetry**

In [ ]:
structure_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        StructureBrowserWidget(title="AiiDA database"),
    ],
    storable=False,
    node_class='CifData')
display(structure_widget)

## Step 2: Select executable

In [ ]:
code_dropdown = CodeDropdown(input_plugin='zeopp.network', path_to_root='../../')
display(code_dropdown)

## Step 3: Compute 

In [ ]:
output = ipw.HTML('')

probe_radius = ipw.BoundedFloatText(
    description='Probe radius [Angstrom]:', 
    min=0.1,
    max=100.0,
    value=1.8,
    layout=ipw.Layout(width="400px"),
    style={"description_width":"initial"}
)

submit_widget = SubmitButtonWidget(NetworkCalculation, on_click_submit)
submit_widget.on_submitted(follow_process)

display(probe_radius, submit_widget, output)